<a href="https://colab.research.google.com/github/vadim-vic/Foundation-ts/blob/main/sandbox/TinyGPT_from_Scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TinyGPT from Scratch

## 1. Setup & toy dataset

We’ll start with a tiny character-level dataset.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# Toy dataset: small text
text = "hello world"
chars = sorted(set(text))
stoi = {ch:i for i,ch in enumerate(chars)}
itos = {i:ch for ch,i in stoi.items()}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)
vocab_size = len(chars)
print("Vocab:", chars)

Vocab: [' ', 'd', 'e', 'h', 'l', 'o', 'r', 'w']


## 2. Batch maker

We need training samples (input → target).

In [ ]:
def get_batch(block_size=4, batch_size=8):
    ix = torch.randint(len(data)-block_size-1, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch()
print(xb[0], "->", yb[0])

tensor([4, 5, 0, 7]) -> tensor([5, 0, 7, 5])


## 3. Self-Attention Head

The heart of GPT: scaled dot-product attention.

In [ ]:
class Head(nn.Module):
    def __init__(self, head_size, n_embd, block_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.tril = torch.tril(torch.ones(block_size, block_size))
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        B, T, C = x.shape
        k = self.key(x)     # (B,T,hs)
        q = self.query(x)   # (B,T,hs)
        wei = q @ k.transpose(-2,-1) * C**-0.5
        wei = wei.masked_fill(self.tril[:T,:T]==0, float('-inf'))
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)
        v = self.value(x)
        out = wei @ v
        return out


## 4. Transformer Block

A GPT block = multi-head attention + MLP + residuals.

In [ ]:
class Block(nn.Module):
    def __init__(self, n_embd, n_head, block_size):
        super().__init__()
        head_size = n_embd // n_head
        self.sa = nn.ModuleList([Head(head_size, n_embd, block_size) for _ in range(n_head)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.mlp = nn.Sequential(
            nn.Linear(n_embd, 4*n_embd),
            nn.ReLU(),
            nn.Linear(4*n_embd, n_embd),
        )
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.proj(torch.cat([h(x) for h in self.sa], dim=-1))
        x = self.ln1(x)
        x = x + self.mlp(x)
        x = self.ln2(x)
        return x


## 5. The GPT Model

Tie it all together.

In [ ]:
class TinyGPT(nn.Module):
    def __init__(self, vocab_size, n_embd=32, block_size=8, n_head=4, n_layer=2):
        super().__init__()
        self.token_emb = nn.Embedding(vocab_size, n_embd)
        self.pos_emb = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head, block_size) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd)
        self.head = nn.Linear(n_embd, vocab_size)

        self.block_size = block_size

    def forward(self, idx, targets=None):
        B, T = idx.shape
        tok = self.token_emb(idx)
        pos = self.pos_emb(torch.arange(T, device=idx.device))
        x = tok + pos
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.head(x)

        if targets is None:
            loss = None
        else:
            loss = F.cross_entropy(logits.view(-1, vocab_size), targets.view(-1))
        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -self.block_size:]
            logits, _ = self(idx_cond)
            probs = F.softmax(logits[:, -1, :], dim=-1)
            next_id = torch.multinomial(probs, num_samples=1)
            idx = torch.cat([idx, next_id], dim=1)
        return idx


## 6. Training

Just a quick run to see it learn.*italicised text*

In [ ]:
model = TinyGPT(vocab_size)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)

for step in range(200):  # try 2000+ for better results
    xb, yb = get_batch()
    logits, loss = model(xb, yb)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if step % 50 == 0:
        print(step, loss.item())


0 2.1965808868408203
50 0.2806495130062103
100 0.12873832881450653
150 0.08644482493400574


## 7. Generate Text

Let’s see if it can speak.

In [ ]:
context = torch.zeros((1,1), dtype=torch.long)  # start token = "h"
print(decode(model.generate(context, max_new_tokens=50)[0].tolist()))


 worllo wo wo worlrlorldrlo wo worlrlo wo wrlrlrlor
